In [ ]:
from flask import Flask, request, render_template_string
from transformers import BertTokenizer, BertForSequenceClassification
from google.colab import drive

# Mount Google Drive to load the fine-tuned model
drive.mount('/content/drive')

# Load the fine-tuned model and tokenizer
model_path = '/content/drive/MyDrive/anlp-sentiment-analysis-bert'
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

# Define sentiment labels (3 classes: negative, neutral, positive)
sentiment_labels = {0: "Negative", 1: "Neutral", 2: "Positive"}

# Flask app initialization
app = Flask("sentiment_analysis_app")

# HTML Template for the UI
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Sentiment Analysis</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            text-align: center;
            margin-top: 50px;
        }
        .container {
            max-width: 500px;
            margin: auto;
            padding: 20px;
            border: 1px solid #ddd;
            border-radius: 8px;
            box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
        }
        input[type="text"] {
            width: 80%;
            padding: 10px;
            margin-bottom: 15px;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        button {
            padding: 10px 20px;
            background-color: #28a745;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
        button:hover {
            background-color: #218838;
        }
        .result {
            margin-top: 20px;
            font-size: 18px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Sentiment Analysis</h1>
        <form method="POST">
            <input type="text" name="sentence" placeholder="Enter a sentence" required />
            <button type="submit">Analyze Sentiment</button>
        </form>
        {% if sentence is not none %}
            <div class="result">
                <p><strong>Sentence:</strong> "{{ sentence }}"</p>
                <p><strong>Predicted Sentiment:</strong> {{ sentiment }}</p>
            </div>
        {% endif %}
    </div>
</body>
</html>
"""

# Flask route to render the web app and handle prediction
@app.route("/", methods=["GET", "POST"])
def sentiment_analysis():
    sentiment = None
    sentence = None

    try:
        if request.method == "POST":
            # Get the sentence input from the form
            sentence = request.form["sentence"]

            # Tokenize the input sentence
            inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)

            # Perform inference using the fine-tuned model
            outputs = model(**inputs)
            predicted_class = outputs.logits.argmax().item()

            # Convert predicted class to a human-readable sentiment label
            sentiment = sentiment_labels.get(predicted_class, "Unknown")
    except Exception as e:
        return f"An error occurred during sentiment analysis: {str(e)}", 500

    # Render the template with the predicted sentiment
    return render_template_string(HTML_TEMPLATE, sentiment=sentiment, sentence=sentence)

# Allow Flask app to be accessed from Colab
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

# Run the Flask app
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
https://5h1g6m7jwbu-496ff2e9c6d22116-5000-colab.googleusercontent.com/
 * Serving Flask app 'sentiment_analysis_app'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Nov/2024 13:26:05] "GET /?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2024 13:26:05] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -
